In [2]:
#Assignment 11#
#My computer was beginning to over heat so I had to cut it short from 20 examples.#
import keras
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))
maxlen = 60
step = 3

sentences = []

next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

print('Number of sequences:', len(sentences))

chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)
import random
import sys

for epoch in range(1, 60):
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)


Corpus length: 600893
Number of sequences: 200278
Unique characters: 57
Vectorization...
epoch 1
1565/1565 [==============================] - 192s 122ms/step - loss: 2.2701
--- Generating with seed: "
above the phase in which he was actuated only by considerat"
------ temperature: 0.2

above the phase in which he was actuated only by consideration of the self the self-the self-some of the wart of the has the sense the self the sense to a to be the would the sime the will of the will of the sertions of the sert of the fall the will the complies and in the complicion of the has the self-the more has to the has the complicion of the more of the compless of the has the sertiness of the compleass of the chars of the seast the man and in the ------ temperature: 0.5
 the compleass of the chars of the seast the man and in the fain or for the ears of consist in the batiges to gardation of chates of more the has the comparable desired and self here ares them or whate somethish--this old and comp

<ipython-input-2-075ad67f7c3a>:45: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


hdpuserness, with its putrianly hen have matadiniss, himress, precisely in his value, epoch 18
 690/1565 [============>.................] - ETA: 1:47 - loss: 1.3132

KeyboardInterrupt: 